In [1]:
import os
import pickle
import numpy as np
import pandas as pd

import torch
from torch import nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid

from sklearn import linear_model
from sklearn.model_selection import StratifiedShuffleSplit

from PIL import Image

In [2]:
class PawpularityDataset(Dataset):
    def __init__(self, main_dir, imgs, labels, meta, transform):
        self.main_dir = main_dir
        self.transform = transform
        self.all_imgs = imgs
        self.labels = labels
        self.meta = meta

    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.all_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image, self.meta[idx], self.labels[idx]

def load_imgs(folder, batch_size=64, val_size=0):
    # fetch filenames
    main_dir = '../input/petfinder-pawpularity-score/{}'.format(folder)
    img_paths = os.listdir(main_dir)
    meta_data = pd.read_csv("../input/petfinder-pawpularity-score/{}.csv".format(folder))
    if folder == 'test':
        features = meta_data.columns[1:]
    else:
        features = meta_data.columns[1:-1]
    
    # fetch labels
    if folder == 'train':
        labels_dict = dict()
        for i in range(len(meta_data)):
            labels_dict[meta_data['Id'][i]] = meta_data['Pawpularity'][i]

        labels = list()
        for img in img_paths:
            labels.append(labels_dict[img.split('.')[0]])
    else:
        labels = [-1 for i in range(len(img_paths))]
    
    # fetch meta data
    meta_dict = dict()
    meta = meta_data[features].to_numpy()
    for i in range(len(meta_data)):
        meta_dict[meta_data['Id'][i]] = meta[i]
    meta = list()
    for img in img_paths:
        meta.append(meta_dict[img.split('.')[0]])
    
    img_paths = np.array(img_paths)
    labels = np.array(labels)
    meta = np.array(meta)
    
    # split if test_size > 0
    if val_size > 0:
        inds = [i for i in range(len(img_paths))]
        np.random.shuffle(inds)
        split_ind = int(len(inds) * val_size)
        train_inds = inds[split_ind:]
        val_inds = inds[:split_ind]

    # declare preprocess (add augmentation here)
    train_preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    val_preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # construct dataset and dataloader    
    if folder == 'test':
        dataset = PawpularityDataset(main_dir=main_dir, imgs=img_paths, labels=labels, meta=meta, transform=val_preprocess)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        
        return dataloader
    else:
        if val_size == 0:
            train_dataset = PawpularityDataset(main_dir=main_dir, imgs=img_paths, labels=labels, meta=meta, transform=train_preprocess)
            train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False) # change shuflle here if do not wanna shuffle
            return train_dataloader
        
        train_dataset = PawpularityDataset(main_dir=main_dir, imgs=img_paths[train_inds], labels=labels[train_inds], meta=meta[train_inds], transform=train_preprocess)
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # change shuflle here if do not wanna shuffle
        
        val_dataset = PawpularityDataset(main_dir=main_dir, imgs=img_paths[val_inds], labels=labels[val_inds], meta=meta[val_inds], transform=val_preprocess)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        return train_dataloader, val_dataloader

In [3]:
class Regressor(nn.Module):
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
                
        self.fc_liner = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, out_size),
            nn.Softplus(),
        )
    
    def forward(self, x):
        return self.fc_liner(x).squeeze()
    
    def predict(self, x):
        return self.forward(x)

class FineTuneModel(nn.Module):
    def __init__(self, latent_size, hidden_size, out_size, device):
        super().__init__()
        self.device = device

        # load pretrained model
        with open('../input/resnet18/resnet18.pkl', 'rb') as f:
            pretrain_model = pickle.load(f)
            self.pretrain_feat = nn.Sequential(*(list(pretrain_model.children())[:-1]))
        
        # make the model fine-tuned
        for param in self.pretrain_feat.parameters():
            param.requires_grad = True
        
        # construct the final output layer(s)
        self.regressor = Regressor(latent_size, hidden_size, out_size)
    
    def forward(self, x, meta):
        feat_out = self.pretrain_feat(x).squeeze()
        # do something with meta data
        N, D = feat_out.shape
        N, M = meta.shape
        out = torch.zeros((N, D+M)).to(self.device)
        out[:, :D] = feat_out
        out[:, D: ] = meta
        return self.regressor(out).squeeze()
    
    def predict(self, x, meta):
        return self.forward(x, meta)

In [4]:
# use gpu if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device:', device)

# laod trained model
print('Load well-trained model...')
latent_out = 524
hidden_size = 4096
model = FineTuneModel(latent_out, hidden_size, 1, device).to(device)
model.load_state_dict(torch.load('../input/pawpularity-resnet18-2layer-mlp/resnet18_2layer_mlp.model'))
model.eval()

print('Num params:', sum(p.numel() for p in model.parameters() if p.requires_grad))

device: cpu
Load well-trained model...
Num params: 21719617


In [5]:
# load test data
print('Construct dataloaders...')
test_dataloader = load_imgs('test', batch_size=128, val_size=0)

# iterate over test data
print('Making predictions...')
test_pred = list()
for test_images, test_meta, test_labels in test_dataloader:
    test_images = test_images.to(device).float()
    test_meta = test_meta.to(device).float()

    # forward
    test_pred += model.predict(test_images, test_meta).cpu().detach().numpy().tolist()

# write to file
print('Write to file...')
ids = pd.read_csv("../input/petfinder-pawpularity-score/{}.csv".format('test'))['Id']
output = pd.DataFrame({"Id": ids, "Pawpularity": test_pred})
output.to_csv('submission.csv', index = False)

Construct dataloaders...
Making predictions...
Write to file...


In [6]:
# check output
output_df = pd.read_csv('submission.csv')
output_df

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,14.408893
1,43a2262d7738e3d420d453815151079e,17.695114
2,4e429cead1848a298432a0acad014c9d,16.995123
3,80bc3ccafcc51b66303c2c263aa38486,14.986017
4,8f49844c382931444e68dffbe20228f4,16.237005
5,b03f7041962238a7c9d6537e22f9b017,15.443115
6,c978013571258ed6d4637f6e8cc9d6a3,15.800261
7,e0de453c1bffc20c22b072b34b54e50f,14.397794
